#### Executing this notebook depends on the Delta tables from project 1 being saved on DBFS!
In project 1, make sure to run chapters 1, 2, 3 fully to save the Delta tables.

### Read data and extract relevant features

In [0]:
import pyspark.sql.functions as F
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [0]:
# Read the full dblp fact table.
dblp_df = (spark
           .read.load(f'dbfs:/user/dblpv13/dblp_full')
           .filter(F.col('Lang') == 'en')) # Keep only papers in English

In [0]:
# Keep just the relevant features: FOS, Keywords, Title, Abstract.
data = dblp_df.select('FOS', 'Keywords', 'Title', 'Abstract')

# Drop all rows where any value is null.
data = data.na.drop()

# Keep only rows where title and abstract are longer than 1 word.
data = (data.filter( (F.size(F.split(data.Title, ' ')) > 1) & 
                     (F.size(F.split(data.Abstract, ' ')) > 1) ))

# Remove all empty (null or '') values from the FOS and Keywords arrays.
data = (data
        .withColumn('FOS', F.expr('filter(FOS, x -> x is not null)'))
        .withColumn('FOS', F.expr('filter(FOS, x -> length(x) > 0)'))
        .withColumn('Keywords', F.expr('filter(Keywords, x -> x is not null)'))
        .withColumn('Keywords', F.expr('filter(Keywords, x -> length(x) > 0)')))

logger.info(f'Our data has {data.count()} samples.')

display(data.limit(10))

INFO:py4j.java_gateway:Received command  on object id 
INFO:py4j.java_gateway:Closing down callback connection
INFO:__main__:Our data has 1356788 samples.


FOS,Keywords,Title,Abstract
"List(Logic synthesis, Topology, Digital electronics, Boolean circuit, Sequential logic, Pass transistor logic, Logic optimization, Algorithm, Register-transfer level, High-level verification, Mathematics)","List(combinational circuits, formal verification, logic design, logic testing, network topology, SAT, combinational circuits, fixed circuit topology, formal verification, logic synthesis, logic verification, minimum ECO, net lists, test vector generation method)",Logic synthesis and verification on fixed topology,"We discuss about logic synthesis and formal verification of combinational circuits mapped to a given fixed topology. Here “fixed topology” means that circuit structures in terms of net lists except for gate/cell types are fixed in advance. That is, for logic synthesis, what should be generated are the types of gates/cells (or simply gates) in the circuits, and all the others are prefixed before synthesis. As the circuit topology is fixed in advance, placement and routing could be shared among different designs and minimum ECO can be realized by keeping the same layout. Also, we can show that we do not need many test vectors in order to guarantee 100% correctness of such synthesis. Small numbers of test vectors, such as only 100 test vectors for 30 input circuits, are sufficient to test if the circuits behave correctly for all possible input value combinations. That is, very efficient formal verification can be realized through simulations with small numbers of test vectors. We present SAT based implementation of the synthesis and a test vector generation method with preliminary but encouraging experimental results."
"List(Detection theory, Computer science, Inference, Algorithm, Artificial intelligence, Fuse (electrical), Machine learning, Recursion)","List(adaptive signal detection, compressed sensing, multidimensional signal processing, signal reconstruction, adaptive sequential compressive signal detection, compressive gain, multidimensional phenomena, nonadaptive strategy, recursive sparse reconstruction algorithm, sequential reconstruction, sparse signal detection, stopping criterion)",Adaptive sequential compressive detection,"Sparsity is at the heart of numerous applications dealing with multidimensional phenomena with low-information content. The primary question that this work investigates is whether, and how much, further compressive gains could be achieved if the goal of the inference task does not require exact reconstruction of the underlying signal. In particular, if the goal is to detect the existence of a sparse signal in noise, it is shown that the number of measurements can be reduced. In contrast to prior work, which considered non-adaptive strategies, a sequential adaptive approach for compressed signal detection is proposed. The key insight is that the decision can be made as soon as a stopping criterion is met during sequential reconstructions. Two sources of performance gains are studied, namely, compressive gains due to adaptation, and computational gains via recursive sparse reconstruction algorithms that fuse newly acquired measurements and previous signal estimates."
"List(Model order selection, Subspace topology, Computer science, Ordinal number, Ordinal data, Matrix decomposition, Speech recognition, Language acquisition, Artificial intelligence, Non-negative matrix factorization, Robot, Machine learning)","List(acoustic signal processing, audio user interfaces, human-robot interaction, intelligent robots, learning (artificial intelligence), matrix decomposition, CSNMF, accuracy improvement, acoustic data augmentation, automatic relevance, command execution, command meaning representation, constrained subspace NMF, grounding information, learning rate improvement, machine learning algorithm, model order selection, ordinal structure, ordinal word acquisition, robot learning, semantic labels, spoken utterances, vocal interface, weakly-supervised NMF, weakly-s

In [0]:
!pip install spacy

     |██████████████████           

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 9.9 MB 40.7 MB/s 
     |████████████████████████████████| 58 kB 9.7 MB/s 
     |████████████████████████████████| 96 kB 9.3 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
!python -m spacy download en_core_web_sm

     |█████████                       | 3.

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 12.8 MB 1.4 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [0]:
import spacy
import re
import string
from pyspark.sql.types import StringType, ArrayType
from collections import Counter

In [0]:
nlp = spacy.load("en_core_web_sm")

In [0]:
def preprocess(text):
    # remove punctuation
    text = re.sub('[^\w\s]', ' ', text)
    # tokenize text
    text_tokens = text.split(' ')
    text_tokens = [word for word in text_tokens if word != '']
    # remove stopwords
    stopwords = nlp.Defaults.stop_words
    tokens = [word for word in text_tokens if not word in stopwords]
    # lowercase and remove custom stopwords
    custom_stopwords = ['doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure','rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'Elsevier', 'PMC', 'CZI', 'www']
    tokens = [token.lower() for token in tokens if token not in custom_stopwords]
    return tokens

preprocessUDF = udf(preprocess, ArrayType(StringType()))

def preprocess_list(arr):
    text = ' '.join(arr)
    # remove punctuation
    text = re.sub('[^\w\s]', ' ', text)
    # tokenize text
    text_tokens = text.split(' ')
    text_tokens = [word for word in text_tokens if word != '']
    # remove stopwords
    stopwords = nlp.Defaults.stop_words
    tokens = [word for word in text_tokens if not word in stopwords]
    # lowercase and remove custom stopwords
    custom_stopwords = ['doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure','rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'medrxiv', 'license', 'fig', 'Elsevier', 'PMC', 'CZI', 'www']
    tokens = [token.lower() for token in tokens if token not in custom_stopwords]
    return tokens

preprocess_listUDF = udf(preprocess_list, ArrayType(StringType()))

In [0]:
processed_data = data.withColumn("FOS", preprocess_listUDF("FOS")) \
                     .withColumn("Keywords", preprocess_listUDF("Keywords")) \
                     .withColumn("Title", preprocessUDF("Title")) \
                     .withColumn("Abstract", preprocessUDF("Abstract"))

In [0]:
display(processed_data.limit(10))

FOS,Keywords,Title,Abstract
"List(logic, synthesis, topology, digital, electronics, boolean, circuit, sequential, logic, pass, transistor, logic, logic, optimization, algorithm, register, transfer, level, high, level, verification, mathematics)","List(combinational, circuits, formal, verification, logic, design, logic, testing, network, topology, sat, combinational, circuits, fixed, circuit, topology, formal, verification, logic, synthesis, logic, verification, minimum, eco, net, lists, test, vector, generation, method)","List(logic, synthesis, verification, fixed, topology)","List(we, discuss, logic, synthesis, formal, verification, combinational, circuits, mapped, given, fixed, topology, here, fixed, topology, means, circuit, structures, terms, net, lists, gate, cell, types, fixed, advance, that, logic, synthesis, generated, types, gates, cells, simply, gates, circuits, prefixed, synthesis, as, circuit, topology, fixed, advance, placement, routing, shared, different, designs, minimum, eco, realized, keeping, layout, also, need, test, vectors, order, guarantee, 100, correctness, synthesis, small, numbers, test, vectors, 100, test, vectors, 30, input, circuits, sufficient, test, circuits, behave, correctly, possible, input, value, combinations, that, efficient, formal, verification, realized, simulations, small, numbers, test, vectors, we, present, sat, based, implementation, synthesis, test, vector, generation, method, preliminary, encouraging, experimental, results)"
"List(detection, theory, computer, science, inference, algorithm, artificial, intelligence, fuse, electrical, machine, learning, recursion)","List(adaptive, signal, detection, compressed, sensing, multidimensional, signal, processing, signal, reconstruction, adaptive, sequential, compressive, signal, detection, compressive, gain, multidimensional, phenomena, nonadaptive, strategy, recursive, sparse, reconstruction, algorithm, sequential, reconstruction, sparse, signal, detection, stopping, criterion)","List(adaptive, sequential, compressive, detection)","List(sparsity, heart, numerous, applications, dealing, multidimensional, phenomena, low, information, content, the, primary, question, work, investigates, compressive, gains, achieved, goal, inference, task, require, exact, reconstruction, underlying, signal, in, particular, goal, detect, existence, sparse, signal, noise, shown, number, measurements, reduced, in, contrast, prior, work, considered, non, adaptive, strategies, sequential, adaptive, approach, compressed, signal, detection, proposed, the, key, insight, decision, soon, stopping, criterion, met, sequential, reconstructions, two, sources, performance, gains, studied, compressive, gains, adaptation, computational, gains, recursive, sparse, reconstruction, algorithms, fuse, newly, acquired, measurements, previous, signal, estimates)"
"List(model, order, selection, subspace, topology, computer, science, ordinal, number, ordinal, data, matrix, decomposition, speech, recognition, language, acquisition, artificial, intelligence, non, negative, matrix, factorization, robot, machine, learning)","List(acoustic, signal, processing, audio, user, interfaces, human, robot, interaction, intelligent, robots, learning, artificial, intelligence, matrix, decomposition, csnmf, accuracy, improvement, acoustic, data, augmentation, automatic, relevance, command, execution, command, meaning, representation, constrained, subspace, nmf, grounding, information, learning, rate, improvement, machine, learning, algorithm, model, order, selection, ordinal, structure, ordinal, word, acquisition, robot, learning, semantic, labels, spoken, utterances, vocal, interface, weakly, supervised, nmf, weakly, supervised, nonnegative, matrix, factorization, automatic, relevance, determination, ard, language, acquisition, machine, learning, nonnegative, matrix, factorization, nmf, ordinal, data)","List(acquisition, ordinal, words, weakly, supervised, nmf)","List(this, paper, issues, design, voca

Combine the lists of each row into one in a new column called 'words'.

In [0]:
def combine_lists(fos, keywords, title, abstract):
    list = []
    for word in fos:
        list.append(word)
    for word in keywords:
        list.append(word)
    for word in title:
        list.append(word)
    for word in abstract:
        list.append(word)
    return list

combine_listsUDF = udf(combine_lists, ArrayType(StringType()))

In [0]:
processed_data = processed_data.withColumn("words", combine_listsUDF("FOS", "Keywords", "Title", "Abstract"))

In [0]:
display(processed_data.limit(10))

FOS,Keywords,Title,Abstract,words
"List(logic, synthesis, topology, digital, electronics, boolean, circuit, sequential, logic, pass, transistor, logic, logic, optimization, algorithm, register, transfer, level, high, level, verification, mathematics)","List(combinational, circuits, formal, verification, logic, design, logic, testing, network, topology, sat, combinational, circuits, fixed, circuit, topology, formal, verification, logic, synthesis, logic, verification, minimum, eco, net, lists, test, vector, generation, method)","List(logic, synthesis, verification, fixed, topology)","List(we, discuss, logic, synthesis, formal, verification, combinational, circuits, mapped, given, fixed, topology, here, fixed, topology, means, circuit, structures, terms, net, lists, gate, cell, types, fixed, advance, that, logic, synthesis, generated, types, gates, cells, simply, gates, circuits, prefixed, synthesis, as, circuit, topology, fixed, advance, placement, routing, shared, different, designs, minimum, eco, realized, keeping, layout, also, need, test, vectors, order, guarantee, 100, correctness, synthesis, small, numbers, test, vectors, 100, test, vectors, 30, input, circuits, sufficient, test, circuits, behave, correctly, possible, input, value, combinations, that, efficient, formal, verification, realized, simulations, small, numbers, test, vectors, we, present, sat, based, implementation, synthesis, test, vector, generation, method, preliminary, encouraging, experimental, results)","List(logic, synthesis, topology, digital, electronics, boolean, circuit, sequential, logic, pass, transistor, logic, logic, optimization, algorithm, register, transfer, level, high, level, verification, mathematics, combinational, circuits, formal, verification, logic, design, logic, testing, network, topology, sat, combinational, circuits, fixed, circuit, topology, formal, verification, logic, synthesis, logic, verification, minimum, eco, net, lists, test, vector, generation, method, logic, synthesis, verification, fixed, topology, we, discuss, logic, synthesis, formal, verification, combinational, circuits, mapped, given, fixed, topology, here, fixed, topology, means, circuit, structures, terms, net, lists, gate, cell, types, fixed, advance, that, logic, synthesis, generated, types, gates, cells, simply, gates, circuits, prefixed, synthesis, as, circuit, topology, fixed, advance, placement, routing, shared, different, designs, minimum, eco, realized, keeping, layout, also, need, test, vectors, order, guarantee, 100, correctness, synthesis, small, numbers, test, vectors, 100, test, vectors, 30, input, circuits, sufficient, test, circuits, behave, correctly, possible, input, value, combinations, that, efficient, formal, verification, realized, simulations, small, numbers, test, vectors, we, present, sat, based, implementation, synthesis, test, vector, generation, method, preliminary, encouraging, experimental, results)"
"List(detection, theory, computer, science, inference, algorithm, artificial, intelligence, fuse, electrical, machine, learning, recursion)","List(adaptive, signal, detection, compressed, sensing, multidimensional, signal, processing, signal, reconstruction, adaptive, sequential, compressive, signal, detection, compressive, gain, multidimensional, phenomena, nonadaptive, strategy, recursive, sparse, reconstruction, algorithm, sequential, reconstruction, sparse, signal, detection, stopping, criterion)","List(adaptive, sequential, compressive, detection)","List(sparsity, heart, numerous, applications, dealing, multidimensional, phenomena, low, information, content, the, primary, question, work, investigates, compressive, gains, achieved, goal, inference, task, require, exact, reconstruction, underlying, signal, in, particular, goal, detect, existence, sparse, signal, noise, shown, number, measurements, reduced, in, contrast, prior, work, considered, non, adaptive, strategies, sequential, adaptive, approach, compressed, signal, detection, pro

Vectorization of the data.

In [0]:
from pyspark.ml.feature import HashingTF, IDF

In [0]:
ht = HashingTF(inputCol="words", outputCol="raw_features")
tf_vectors = ht.transform(processed_data.select("words"))
display(tf_vectors.limit(10))

words,raw_features
"List(logic, synthesis, topology, digital, electronics, boolean, circuit, sequential, logic, pass, transistor, logic, logic, optimization, algorithm, register, transfer, level, high, level, verification, mathematics, combinational, circuits, formal, verification, logic, design, logic, testing, network, topology, sat, combinational, circuits, fixed, circuit, topology, formal, verification, logic, synthesis, logic, verification, minimum, eco, net, lists, test, vector, generation, method, logic, synthesis, verification, fixed, topology, we, discuss, logic, synthesis, formal, verification, combinational, circuits, mapped, given, fixed, topology, here, fixed, topology, means, circuit, structures, terms, net, lists, gate, cell, types, fixed, advance, that, logic, synthesis, generated, types, gates, cells, simply, gates, circuits, prefixed, synthesis, as, circuit, topology, fixed, advance, placement, routing, shared, different, designs, minimum, eco, realized, keeping, layout, also, need, test, vectors, order, guarantee, 100, correctness, synthesis, small, numbers, test, vectors, 100, test, vectors, 30, input, circuits, sufficient, test, circuits, behave, correctly, possible, input, value, combinations, that, efficient, formal, verification, realized, simulations, small, numbers, test, vectors, we, present, sat, based, implementation, synthesis, test, vector, generation, method, preliminary, encouraging, experimental, results)","Map(vectorType -> sparse, length -> 262144, indices -> List(1928, 3910, 4385, 4971, 7075, 7441, 8732, 19210, 26179, 31159, 31903, 33358, 35091, 36370, 40875, 42884, 44673, 46002, 48448, 50208, 53569, 63654, 67058, 71637, 72462, 76626, 80058, 82541, 82555, 82919, 83161, 94311, 94438, 98586, 98877, 100511, 101559, 104313, 108615, 109802, 111606, 113359, 116352, 118366, 122417, 123499, 125984, 136020, 141652, 146957, 147136, 149980, 150319, 156084, 156590, 157971, 158069, 170543, 172640, 177425, 177426, 178129, 179770, 182404, 183375, 184280, 188762, 194240, 198589, 201547, 202290, 213504, 218319, 219879, 220850, 223540, 227860, 229979, 230277, 231108, 237834, 244132, 245522, 252535, 256101, 257347, 257750), values -> List(2.0, 1.0, 1.0, 1.0, 4.0, 1.0, 4.0, 7.0, 3.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 8.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 7.0, 6.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 4.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 6.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 7.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 11.0, 1.0, 1.0, 2.0, 2.0, 1.0))"
"List(detection, theory, computer, science, inference, algorithm, artificial, intelligence, fuse, electrical, machine, learning, recursion, adaptive, signal, detection, compressed, sensing, multidimensional, signal, processing, signal, reconstruction, adaptive, sequential, compressive, signal, detection, compressive, gain, multidimensional, phenomena, nonadaptive, strategy, recursive, sparse, reconstruction, algorithm, sequential, reconstruction, sparse, signal, detection, stopping, criterion, adaptive, sequential, compressive, detection, sparsity, heart, numerous, applications, dealing, multidimensional, phenomena, low, information, content, the, primary, question, work, investigates, compressive, gains, achieved, goal, inference, task, require, exact, reconstruction, underlying, signal, in, particular, goal, detect, existence, sparse, signal, noise, shown, number, measurements, reduced, in, contrast, prior, work, considered, non, adaptive, strategies, sequential, adaptive, approach, compressed, signal, detection, proposed, the, key, insight, decision, soon, stopping, criterion, met, sequential, reconstructions, two, sources, performance, gains, studied, compressive, gains, adaptation, computational, gains, recursive, sparse, reconstruction, algorithms, fuse, newly, acquired, measurements, previous, signal, estimates)","Map(vectorType -> sparse

In [0]:
idf = IDF(inputCol="raw_features", outputCol="features", minDocFreq=5)
idf_model = idf.fit(tf_vectors)
tfIdf_vectors = idf_model.transform(tf_vectors).cache()

In [0]:
display(tfIdf_vectors.limit(10))

words,raw_features,features
"List(logic, synthesis, topology, digital, electronics, boolean, circuit, sequential, logic, pass, transistor, logic, logic, optimization, algorithm, register, transfer, level, high, level, verification, mathematics, combinational, circuits, formal, verification, logic, design, logic, testing, network, topology, sat, combinational, circuits, fixed, circuit, topology, formal, verification, logic, synthesis, logic, verification, minimum, eco, net, lists, test, vector, generation, method, logic, synthesis, verification, fixed, topology, we, discuss, logic, synthesis, formal, verification, combinational, circuits, mapped, given, fixed, topology, here, fixed, topology, means, circuit, structures, terms, net, lists, gate, cell, types, fixed, advance, that, logic, synthesis, generated, types, gates, cells, simply, gates, circuits, prefixed, synthesis, as, circuit, topology, fixed, advance, placement, routing, shared, different, designs, minimum, eco, realized, keeping, layout, also, need, test, vectors, order, guarantee, 100, correctness, synthesis, small, numbers, test, vectors, 100, test, vectors, 30, input, circuits, sufficient, test, circuits, behave, correctly, possible, input, value, combinations, that, efficient, formal, verification, realized, simulations, small, numbers, test, vectors, we, present, sat, based, implementation, synthesis, test, vector, generation, method, preliminary, encouraging, experimental, results)","Map(vectorType -> sparse, length -> 262144, indices -> List(1928, 3910, 4385, 4971, 7075, 7441, 8732, 19210, 26179, 31159, 31903, 33358, 35091, 36370, 40875, 42884, 44673, 46002, 48448, 50208, 53569, 63654, 67058, 71637, 72462, 76626, 80058, 82541, 82555, 82919, 83161, 94311, 94438, 98586, 98877, 100511, 101559, 104313, 108615, 109802, 111606, 113359, 116352, 118366, 122417, 123499, 125984, 136020, 141652, 146957, 147136, 149980, 150319, 156084, 156590, 157971, 158069, 170543, 172640, 177425, 177426, 178129, 179770, 182404, 183375, 184280, 188762, 194240, 198589, 201547, 202290, 213504, 218319, 219879, 220850, 223540, 227860, 229979, 230277, 231108, 237834, 244132, 245522, 252535, 256101, 257347, 257750), values -> List(2.0, 1.0, 1.0, 1.0, 4.0, 1.0, 4.0, 7.0, 3.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 8.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 7.0, 6.0, 1.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 4.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 6.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 7.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 11.0, 1.0, 1.0, 2.0, 2.0, 1.0))","Map(vectorType -> sparse, length -> 262144, indices -> List(1928, 3910, 4385, 4971, 7075, 7441, 8732, 19210, 26179, 31159, 31903, 33358, 35091, 36370, 40875, 42884, 44673, 46002, 48448, 50208, 53569, 63654, 67058, 71637, 72462, 76626, 80058, 82541, 82555, 82919, 83161, 94311, 94438, 98586, 98877, 100511, 101559, 104313, 108615, 109802, 111606, 113359, 116352, 118366, 122417, 123499, 125984, 136020, 141652, 146957, 147136, 149980, 150319, 156084, 156590, 157971, 158069, 170543, 172640, 177425, 177426, 178129, 179770, 182404, 183375, 184280, 188762, 194240, 198589, 201547, 202290, 213504, 218319, 219879, 220850, 223540, 227860, 229979, 230277, 231108, 237834, 244132, 245522, 252535, 256101, 257347, 257750), values -> List(15.375382687872822, 4.26701713463519, 4.950904747584705, 4.714559820995535, 14.139118523779945, 3.1645057283813034, 13.639459078138417, 23.207220784132907, 18.502993937257376, 4.690834301243194, 2.7780694414986167, 5.809907298053688, 2.389993140091139, 3.847202332100345, 5.242273396047769, 30.453529876162072, 6.18572769744555, 8.996667457272332, 10.926657074549416, 3.1974665835594953, 5.432514733417768, 4.212255850066686, 3.169088319088505, 2.56025956711943, 3.3233032811028993, 2.205887504517395, 4.957068256258337, 26.72786968675708, 21.985590144854832, 2.0954393866229704, 2.900883114439123, 8.464007494092973, 4.3166327